In [2]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes

import pandas as pd
import numpy as np

# Store excel file locations to variables (change it as per your path to file)
file_2012=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\3. Section 2 Part 1 EMP&INC Time use\2012s2p1_f.csv"
file_2013=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\3. Section 2 Part 1 EMP&INC Time use\2013_s2p1_f.csv"
file_2014=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\3. Section 2 Part 1 EMP&INC Time use\2014_s2p1_f.csv"

# Read excel files
df_2012 = pd.read_csv(file_2012)
df_2013 = pd.read_csv(file_2013)
df_2014 = pd.read_csv(file_2014)


In [3]:
'''
This code block will be used to standardize column names across the years to avoid discrepancies during the merging process.
For example in the roaster data for 2013 rq21 and rq23 are not the same as rq21 and rq23 in 2014 data, but they have the same variable names. Hence, we decide to rename such columns beforehand
We will add the updated name to the mapping dictionaries instead of the original names.

'''

# Rename columns in df_2012
df_2012.rename(columns={
    'hid': 'hid',
    'round': 'round',
    'pid': 'r_pid',
    'S2P1Q2': 's2p1_q2',
    'S2P1Q3': 's2p1_q3',
    'S2P1Q4': 's2p1_q4',
    'S2P1Q5': 's2p1_q5',
    'S2P1Q6': 's2p1_q6',
    'S2P1Q7': 's2p1_q7',
    'S2P1Q8': 's2p1_q8',
    'S2P1Q9': 's2p1_q9',
    'S2P1Q10': 's2p1_q10',
    'S2P1Q11': 's2p1_q11',
    'S2P1Q12': 's2p1_q12',
    'S2P1Q13': 's2p1_q13',
    'S2P1Q14': 's2p1_q14',
    'S2P1Q15': 's2p1_q15',
    'S2P1Q16': 's2p1_q16',
    'PROVINCE_ID': 'P_ID',
    'DISTRICT_ID': 'D_ID',
    'TEHSIL_ID': 'T_ID',
    'UC_ID': 'UC_ID',
    'MAUZA_ID': 'M_ID'
}, inplace=True)


# df_2014 and df_2013 doesn't need renaming as it is the reference

In [5]:
# Updated mappings

mapping_2012 = [
    'hid', 'round', 'r_pid', 's2p1_q2', 's2p1_q3', 's2p1_q4', 's2p1_q5', 's2p1_q6', 's2p1_q7', 's2p1_q8', 's2p1_q9', 's2p1_q10',
    's2p1_q11', 's2p1_q12', 's2p1_q13', 's2p1_q14', 's2p1_q15', 's2p1_q16', None, None, None, None, None, None, None, 'P_ID', 'D_ID',
    'T_ID', 'UC_ID', 'M_ID'
]

mapping_2013 = [
    'hid', 'round', 'r_pid', 's2p1_q2', 's2p1_q3', 's2p1_q4', 's2p1_q5', 's2p1_q6', 's2p1_q7', 's2p1_q8', 's2p1_q9', 
    's2p1_q10', 's2p1_q11', 's2p1_q12', 's2p1_q13', 's2p1_q14', 's2p1_q15', 's2p1_q16', None, None, None, None, None, None,
    None, None, None, None, None, None
]

mapping_2014 = [
    'hid', 'round', 'r_pid', 's2p1_q2', 's2p1_q3', 's2p1_q4', 's2p1_q5', 's2p1_q6', 's2p1_q7', 's2p1_q8', 's2p1_q9', 
    's2p1_q10', 's2p1_q11', 's2p1_q12', 's2p1_q13', 's2p1_q14', 's2p1_q15', 's2p1_q16', 's2p1_q17', 's2p1_q18', 's2p1_q19', 
    's2p1_q20', 's2p1_q21', 's2p1_q22', 's2p1_q23', None, None, None, None, None
]


In [6]:
# Create a list of all possible columns in the correct order
all_columns = []

for col in mapping_2012:
    if col and col not in all_columns:
        all_columns.append(col)   
for col in mapping_2013:
    if col and col not in all_columns:
        all_columns.append(col)
for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [7]:
def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col.strip()  # Remove leading/trailing whitespace
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if ref_col in df.columns:
                    print(f"Appending data for column {ref_col}")
                    if isinstance(df[ref_col], pd.Series):
                        merged_data[ref_col].extend(df[ref_col].tolist())
                    elif isinstance(df[ref_col], pd.DataFrame):
                        print(f"Column {ref_col} is duplicated in DataFrame. Appending data for each duplicate.")
                        for _, series in df[ref_col].items():
                            merged_data[ref_col].extend(series.tolist())
                else:
                    print(f"Column {ref_col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df


In [8]:
# Usage with dataframes and mappings
dfs = [df_2012, df_2013, df_2014]
mappings = [mapping_2012, mapping_2013, mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['Unnamed: 0.1', 'Unnamed: 0', 'hid', 'round', 'r_pid', 's2p1_q2', 's2p1_q3', 's2p1_q4', 's2p1_q5', 's2p1_q6', 's2p1_q7', 's2p1_q8', 's2p1_q9', 's2p1_q10', 's2p1_q11', 's2p1_q12', 's2p1_q13', 's2p1_q14', 's2p1_q15', 's2p1_q16', 'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID']
Appending data for column hid
Appending data for column round
Appending data for column r_pid
Appending data for column s2p1_q2
Appending data for column s2p1_q3
Appending data for column s2p1_q4
Appending data for column s2p1_q5
Appending data for column s2p1_q6
Appending data for column s2p1_q7
Appending data for column s2p1_q8
Appending data for column s2p1_q9
Appending data for column s2p1_q10
Appending data for column s2p1_q11
Appending data for column s2p1_q12
Appending data for column s2p1_q13
Appending data for column s2p1_q14
Appending data for column s2p1_q15
Appending data for column s2p1_q16
Appending data for column P_ID
Appending data for column D_ID
Appending data for colu

In [9]:
# Rename columns for the merged file (if needed)
rename_mapping = {
    'hid': 'HID',
    'round': 'Round',
    'r_pid': 'PID',
    's2p1_q2': 'TU_CookHH_hrs',
    's2p1_q3': 'TU_WCU_hrs',
    's2p1_q4': 'TU_CHH_hrs',
    's2p1_q5': 'TU_HnC_hrs',
    's2p1_q6': 'TU_CW7d_hrs',
    's2p1_q7': 'TU_CFW7d_hrs',
    's2p1_q8': 'TU_WPC7d_hrs',
    's2p1_q9': 'TU_SnCWHH7d_hrs',
    's2p1_q10': 'TU_DC7d_hrs',
    's2p1_q11': 'TU_HHAA7d_hrs',
    's2p1_q12': 'TU_HHNAA7d_hrs',
    's2p1_q13': 'TU_HAW7d_hrs',
    's2p1_q14': 'TU_HNAW7d_hrs',
    's2p1_q15': 'TU_ShopHH7d_hrs',
    's2p1_q16': 'TU_ManHH12m_hrs',
    's2p1_q17': 'TU_TechTrain',
    's2p1_q18': 'TU_TechTrain1_Des',
    's2p1_q19': 'TU_TechTrain1_Code',
    's2p1_q20': 'TU_TechTrain2_Des',
    's2p1_q21': 'TU_TechTrain2_Code',
    's2p1_q22': 'TU_TechTrain1_Dur',
    's2p1_q23': 'TU_TechTrain2_Dur'
}

merged_df.rename(columns=rename_mapping, inplace=True)
'''
# Drop redundant columns
merged_df.drop(merged_df.columns[merged_df.columns.str.contains('Unnamed', case=True)], axis=1, inplace=True)
merged_df.drop(merged_df.columns[merged_df.columns.str.contains(' ', case=False)], axis=1, inplace=True)
'''
# Save the merged dataframe to a CSV file
merged_df.to_csv('1. merged_EnI_Time Use.csv', index=False)
